In [99]:
import numpy as np
import pandas as pd

### Load parameter

In [100]:
sheet = "Problem 3 Instance 1"
file = "OR112-2_hw02_data.xlsx"

m = pd.read_excel(file,usecols="B:B",nrows=1,sheet_name=sheet)
m = int(m.columns.item())
n = pd.read_excel(file,usecols="B:B",skiprows=range(1),nrows=1,sheet_name=sheet)
n = int(n.columns.item())
p = pd.read_excel(file,usecols="B:B",skiprows=range(2),nrows=1,sheet_name=sheet)
p = int(p.columns.item())

d = pd.read_excel(file,usecols="E:"+chr(ord('E')+n-1),skiprows=range(4),nrows=m,sheet_name=sheet)
d = d.to_numpy()

h = pd.read_excel(file,usecols="B:B",skiprows=range(4),nrows=m,sheet_name=sheet)
h = h.to_numpy()

### Variables

In [101]:
x = np.ones(n)
w = [np.min(d[i]) for i in range(m)]

total = sum(h[i]*w[i] for i in range(m)).item()


### Algorithm

In [102]:
def objfunc(h,d,x):
    total = 0
    for i in range(h.shape[0]):
        total += h[i] * min(d[i,x.astype(bool)])
    return total.item()



In [103]:
x = np.ones(n)
for i in range(n-p):
    total = np.zeros(n)
    for j in range(n-i):
        tmp_x = x.copy()
        if tmp_x[j] == 1:
            tmp_x[j] = 0
            total[j] = objfunc(h,d,tmp_x)
    x[np.argmax(total)] = 0
print(x)
print(objfunc(h,d,x))


[1. 1. 1. 0. 0.]
39027
